# <center> Maximum Likelihood Quantum Amplitude Estimation </center>

    Código basado en el paper "Amplitude Estimation without Phase Estimation" de: Yohichi Suzuki, Shumpei Uno, Rudy Raymond, Tomoki Tanaka, Tamiya Onodera y Naoki Yamamoto, https://doi.org/10.1007/s11128-019-2565-2

El enfoque convencional para la estimación de amplitud consiste en utilizar el algoritmo de estimación de fase, el cual conlleva muchas operaciones de amplificación controladas seguidas de una transformada de Fourier en el mundo cuántico. Este procedimiento es difícil de implementar en el escenario actual en el que los recursos cuánticos (procesamiento, qubits...) son límitados.

Próximamente se realizarán dispositivos cuánticos reales con varias centenas de qubits, aunque estos son conocidos como dispositivos cuánticos de escala intermedia ruidosos (NISQ, *Noisy Intermidiate-Scale Quantum*, por sus siglas en inglés) y presentan varias limitaciones prácticas, tanto en el número de operaciones, como en el número de qubits disponibles.

La idea principal es utilizar la estimación de máxima verosimilitud (Maximum Likelihood Estimation) basada en datos de medición producidos por circuitos cuánticos con diferentes números de operaciones. De ahí el nombre del Algoritmo - **MLQAE**

#### En primer lugar, realizaremos un experimento analítico
Un ejemplo simple para encontrar un resultado de forma "tradicional". La integral en un intervalo del $sin(x)$ al cuadrado

$I = \frac{1}{b_{\text{max}}} \int_{0}^{b_{\text{max}}} \sin(x)^2 \, dx$. Que, en el mundo discreto se aproxima como: 

$S = \sum_{x=0}^{2^n-1} p(x) \sin^2\left(\frac{(x + \frac{1}{2}) \cdot b_{\text{max}}}{2^n}\right)$

en donde $p(x) = \frac{1}{2^n}$ es una distribución discreta uniforme de probabilidad.

In [ ]:
import numpy as np #este import es el único que necesitamos para "explicar" esta parte

# definimos la función para realizar la integral del seno^2 
def sin_integral(upper_lim: float) -> float:
    res_continuo = (upper_lim / 2.0 - np.sin(2 * upper_lim) / 4.0)
    
    return res_continuo/upper_lim

#definimos la función para realizar la suma discretizada del seno^2.
def sin_discrete_sum(upper_lim: float, bits_number: int) -> float:
    n_dis = 2**bits_number
    res_discreto = 0.0
    
    for ii in range(n_dis):
        res_discreto += np.sin(upper_lim / n_dis * (ii + (1/2)))**2
        
    res_discreto = res_discreto / n_dis
    return res_discreto

b_max = np.pi / 4
n_bits = 2 #q-bits

mundo_continuo = sin_integral(b_max)
print("El resultado en el mundo continuo es: " + str(mundo_continuo))


mundo_discreto = sin_discrete_sum(b_max, n_bits)
print("El resultado en el mundo discreto es: " + str(mundo_discreto))

## Inicialización del entorno cuántico

In [ ]:
# Importamos las librearías: 

from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit #necesaria para crear bits, qubits y circuitos
from qiskit import execute #necesaria para dar la orden de ejecución
from qiskit import IBMQ #necesaria para llamar a la plataforma IBM Quantum
from qiskit import Aer #necesaria para importar simuladores cuánticos o lanzar ejecuciones en ordenadores reales
from scipy import optimize #optimización matmática --> encontrar el valor = 0 (utilizo opmtimize.brute)
from qiskit_ibm_provider import IBMProvider

import sys, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

IBMProvider.save_account(token="d83e53dfa62683e4eb3ed49770c00c03322d6c921625a94e55cb84c76414b00d915b95732b9e00f1f7502809a7575326ab91316d0eac89acebfd88cc52e483aa", overwrite=True)

print("Realizar simulación en ordenador real o en simulador?")
#job_type = str(input())
job_type = "REAL"


while job_type.upper() not in {"REAL", "SIMULADOR"}:
    print("Por favor, introduce un parámetro válido -> real / simulador")
    job_type = str(input())


if job_type.upper() == "REAL":
    #en caso de querer realizar una ejecución en un ordenador cuántico tenemos dos posibilidades, EXTERNO o CSIC (cátedra UAM / IBM)
    acceso = "EXTERNO"
    Backend_qubits = 20
    print("Acceso seleccionado: "+acceso)
else:
    None


if job_type.upper() == "SIMULADOR":
    provider = IBMProvider()
    backend = provider.get_backend("ibmq_qasm_simulator")
    #backend = Aer.get_backend('qasm_simulator')
    print("Simulador configurado correctamente: "+str(backend))
else:
    if acceso == 'CSIC' :
        # number of qubits to represent the uncertainty ***********************************************************************************************************
        numero_de_qbits_de_incertidumbre = 3
        #**********************************************************************************************************************************************************
        provider = IBMProvider(hub='ibm-q-csic')
        provider.backends()
        from qiskit.providers.ibmq import least_busy
        backend = least_busy(provider.backends(filters=lambda b: b.configuration().n_qubits >= Backend_qubits and
        not b.configuration().simulator and b.status().operational==True))
        print (backend)
        sistema_cuantico = str(backend)
        print("El programa se ejecutará en: "+str(sistema_cuantico))
        print (sistema_cuantico)
    
    else :
        # number of qubits to represent the uncertainty ***********************************************************************************************************
        numero_de_qbits_de_incertidumbre = 3
        #**********************************************************************************************************************************************************
        # Loading your IBM Quantum account(s)
        provider = IBMProvider()
        sistema_cuantico = 'ibmq_manila'
        backend = provider.get_backend(sistema_cuantico)
        print("El programa se ejecutará en: "+str(sistema_cuantico))

## Creación de operadores del algoritmo de Grover y circuito de Amplificación de Amplitud (QAA).

Comenzamos por la inicialización del circuito, un estado "uniforme" sencillo de preparar ${P}$. Representado en el circuito por puertas Hadamard para crear una distribución de probablidad uniforme. 

### Introducción al algoritmo de Grover

El algoritmo de Grover utiliza el operador de Grover $\mathcal{Q}$ para amplificar las amplitudes de los estados buenos:

$$
\mathcal{Q} = \mathcal{A}\mathcal{S_0}\mathcal{A}^\dagger \mathcal{S_f}
$$

Aquí,
- $\mathcal{A}$ es el estado de búsqueda inicial para el algoritmo, que consiste en Hadamards, $H^{\otimes n}$ para la búsqueda de Grover clásica, pero puede ser más elaborado para la amplificación de amplitud.
- $\mathcal{S_0}$ es la reflexión sobre el estado todo 0:

$$
|x\rangle \mapsto \begin{cases} -|x\rangle, &x \neq 0 \\ |x\rangle, &x = 0\end{cases}
$$

- $\mathcal{S_f}$ es el oráculo que aplica:

$$
|x\rangle \mapsto (-1)^{f(x)}|x\rangle
$$

   donde $f(x)$ es 1 si $x$ es un estado bueno y 0 en caso contrario.

En resumen, el algoritmo de Grover aplica diferentes potencias de $\mathcal{Q}$ y después de cada ejecución verifica si se ha encontrado una solución buena.

### Ejecutando el algoritmo de Grover

Para ejecutar el algoritmo de Grover, qiskit tiene su propia clase, en este código definimos una función más abajo que nos permite ganar independencia frente a las actualizaciones que vayan surgiendo. 

In [ ]:
def P(qc, qx):
    """
        Generación de ua distribución de probablidad uniforme
            qc: quantum circuit
            qx: quantum register
            nbit: number of qubits
        The inverse of P = P
    """
    qc.h(qx)

def A(qc, qx, qx_measure, nbit, b_max):
    """
        Preparación del estado, integral de f()
            qc: quantum circuit
            qx: quantum register
            qx:measure: quantum register for measurement
            nbit: number of qubits
            b_max: upper limit of integral            
    """
    qc.ry(b_max / 2**nbit * 2 * 0.5, qx_measure[0])
    
    for i in range(nbit):
        qc.cry(2**i * b_max / 2**nbit * 2, qx[nbit-1-i], qx_measure[0])
               
        
def Ainv(qc, qx, qx_measure, nbit, b_max):
    """
        Puerta inversa de A
            qc: quantum circuit
            qx: quantum register
            qx_measure : quantum register for measurement
            nbit: number of qubits
            b_max: upper limit of integral
    """
    for i in range(nbit)[::-1]:
        qc.cry(-2**i * b_max / 2**nbit * 2, qx[nbit-1-i], qx_measure[0])
     
    qc.ry(-b_max / 2**nbit * 2 * 0.5, qx_measure[0])

def multi_control_NOT(qc, qx, qx_measure, qx_ancilla, nbit):    
    """
        Creación de la puerta cuántica multi control NOT. 
            qc: circuito cuántico
            qx: registro
            qx_measure: registro sobre el que realizaremos la medida
            qx_ancilla: bit ancilla de control. 
            nbit: number of qubits
    """

    if nbit == 1:
        qc.cz(qx[0], qx_measure[0])
        
    elif nbit == 2:
        qc.h(qx_measure[0])
        qc.ccx(qx[0], qx[1], qx_measure[0])
        qc.h(qx_measure[0])
        
    elif nbit > 2.0:
        qc.ccx(qx[0], qx[1], qx_ancilla[0])
        
        for i in range(nbit - 3):
            qc.ccx(qx[i + 2], qx_ancilla[i], qx_ancilla[i + 1])
            
        qc.h(qx_measure[0])
        qc.ccx(qx[nbit - 1], qx_ancilla[nbit - 3], qx_measure[0])
        qc.h(qx_measure[0])
        
        for i in range(nbit - 3)[::-1]:
            qc.ccx(qx[i + 2], qx_ancilla[i], qx_ancilla[i + 1])
            
        qc.ccx(qx[0], qx[1], qx_ancilla[0])


#def reflect(qc, qx, qx_measure, qx_ancilla, nbit, b_max):        
def S_0(qc, qx, qx_measure, qx_ancilla, nbit):
    """
        Operador de reflexion (I - 2|0><0|). Hacemos la reflexión sobre el eje del vector complejo
            qc: quantum circuit
            qx: quantum register
            qx_measure: quantum register for measurement
            qx_ancilla: temporal quantum register for decomposing multi controlled NOT gate
            nbit: number of qubits
            b_max: upper limit of integral
    """
    qc.barrier()
    for i in range(nbit):
        qc.x(qx[i])
    
    qc.x(qx_measure[0])
    #multi_control_NOT(qc, qx, qx_measure, qx_ancilla, nbit, b_max)
    multi_control_NOT(qc, qx, qx_measure, qx_ancilla, nbit)
    qc.x(qx_measure[0])
    
    for i in range(nbit):
        qc.x(qx[i])
    
    qc.barrier()

# This is to implement Grover Operator Q
def Q(qc, qx, qx_measure, qx_ancilla, nbit, b_max):
    """
        El operador de Grover
            qc: quantum circuit
            qx: quantum register
            qx_measure: quantum register for measurement
            qx_ancilla: temporal quantum register for decomposing multi controlled NOT gate
            nbit: number of qubits
            b_max: upper limit of integral
    """
    qc.z(qx_measure[0])
    qc.barrier()
    
    Ainv(qc, qx, qx_measure, nbit, b_max)
    qc.barrier()
    P(qc, qx)

    S_0(qc, qx, qx_measure, qx_ancilla, nbit)
    P(qc, qx)
    qc.barrier()
    A(qc, qx, qx_measure, nbit, b_max)    

### Creación del circuito de grover

In [ ]:
def create_grover_circuit(number_grover_list, nbit, b_max):
    """
        To generate quantum circuits running Grover operators with number of iterations in number_grover_list
            numebr_grover_list: list of number of Grover operators
            nbit: number of qubits (2**nbit = ndiv is the number of discretization in the Monte Carlo integration)
            b_max: upper limit of integral
        Return:
            qc_list: quantum circuits with Grover operators as in number_grover_list
    """
    qc_list = []
    
    for igrover in range(len(number_grover_list)):
        
        qx = QuantumRegister(nbit)
        qx_measure = QuantumRegister(1)
        cr = ClassicalRegister(1)
        if (nbit > 2):
            qx_ancilla = QuantumRegister(nbit - 2)
            qc = QuantumCircuit(qx, qx_ancilla, qx_measure, cr)
        else:
            qx_ancilla = 0
            qc = QuantumCircuit(qx, qx_measure, cr)
            
        P(qc, qx)
        qc.barrier()
        A(qc, qx, qx_measure, nbit, b_max)
        qc.barrier()
        
        for ii in range(number_grover_list[igrover]):
            Q(qc, qx, qx_measure, qx_ancilla, nbit, b_max)
        qc.measure(qx_measure[0], cr[0])
        qc_list.append(qc)
        
        
    return qc_list

## Mostramos el circuito

In [ ]:
qc_list = create_grover_circuit([1], n_bits, b_max)

In [ ]:
qc_list[0].draw("mpl", style="iqx")

## Y ahora creamos una función auxiliar para poder ejecutar "listas" de circuitos cuánticos
- Esta funcion nos da flexibilidad a la hora de incrementar la potencia de $Q$ (meter varios operadores Grover)### Necesitamos una función para ejecutar los circuitos que nos devuelve la función anterior.

In [ ]:
def run_grover(qc_list, number_grover_list, shots_list, backend):
    """
        Run the quantum circuits returned by create_grover_circuit()
            qc_list: list of quantum circuits
            numebr_grover_list: list of number of Grover operators
            shots_list:  list of number of shots
            backend: name of backends
        
        Return:
            hit_list: list of count of obserbving "1" for qc_list
    """
    
    hit_list = []
    for k in range(len(number_grover_list)):
        job = execute(qc_list[k], backend=backend, shots=shots_list[k])
        
        lapse = 0
        interval = 0.00001
        
        time.sleep(interval)
        
        while job.status().name != 'DONE':
            time.sleep(interval)
            lapse += 1
        
        counts = job.result().get_counts(qc_list[k]).get("1", 0)
        
        hit_list.append(counts)
        
    return hit_list


### Etapa de post procesado
#### En esta etapa consegimos obtener nuestro propio $theta$

La función de postprocesamiento combina los resultados de ejecutar circuitos de Grover con varias  iteraciones aplicando MLE. Supongamos que ejecutamos los operadores de Grover ($Q$) ${m_0, m_1,...,m_M}$ veces y, para cada ejecución, lo repetimos $N_k$ veces, de las cuales observamos $h_k$ veces estados "buenos". Dado que la probabilidad de observar estados "buenos" es $\sin^2((2m_k+1)\theta_a)$, la verosimilitud (*likelihood*) de observar $h_k$ veces estados "buenos" es: $L(h, \theta)$

Utilizamos el límite inferior de cramer.rao (ver referencia en: [paper QAE del Google Drive]).


In [ ]:
def CalcErrorCramerRao(M, shot_list, p0, number_grover_list):
    """
        calculate Cramér-Rao lower bound
            M: upper limit of the sum in Fisher information 
            shots_list:  list of number of shots
            p0: the true parameter value to be estimated
            numebr_grover_list: list of number of Grover operators        

        Return:
            square root of Cramér-Rao lower bound:  lower bound on the standard deviation of unbiased estimators
    """
    FisherInfo = 0
    for k in range(M + 1):
        Nk = shot_list[k]
        mk = number_grover_list[k]
        FisherInfo += Nk / (p0 * (1 - p0)) * (2 * mk + 1)**2
    return np.sqrt(1 / FisherInfo)



def calculate_theta(hit_list, number_grover_list, shots_list):
    """
        calculate optimal theta values
            hit_list: list of count of obserbving "1" for qc_list
            numebr_grover_list: list of number of Grover operators        
            shots_list: list of number of shots

        Return:
            thetaCandidate_list: list of optimal theta
    """

    small = 1.e-15  # small valued parameter to avoid zero division
    confidenceLevel = 5  # confidence level to determine the search range

    thetaCandidate_list = []
    a_candidate_list = []
    grover_list = []
    rangeMin = 0.0 + small
    rangeMax = 1.0 - small
    for igrover in range(len(number_grover_list)): #entramos en este bucle utilizando la potencia de grover, no el número (i,e,. 0,1,2,3....)

        def loglikelihood(p):
            ret = 0.0
            theta = np.arcsin(np.sqrt(p))
            
            for n in range(igrover + 1):
                ihit = hit_list[n]
                arg = (2 * number_grover_list[n] + 1) * theta
                ret = ret + 2 * ihit * np.log(np.abs(np.sin(arg))) + 2 * (
                    shots_list[n] - ihit) * np.log(np.abs(np.cos(arg)))
            return -ret

        searchRange = [rangeMin, rangeMax]
        searchResult = optimize.brute(loglikelihood, ranges=[searchRange], finish=None)
        a_candidate = searchResult
        a_candidate_list.append(a_candidate)
        thetaCandidate_list.append(np.arcsin(np.sqrt(a_candidate)))
        grover_list.append(number_grover_list[igrover])
        perror = CalcErrorCramerRao(igrover, shots_list, a_candidate, number_grover_list)
        rangeMax = min(a_candidate+confidenceLevel*perror,1.0-small)
        rangeMin = max(a_candidate-confidenceLevel*perror,0.0+small)

        
        
    return thetaCandidate_list, grover_list, a_candidate_list

In [ ]:
def CalcNumberOracleCalls(M, shot_list, number_grover_list):
    """
        calculate the total number of oracle calls
            M: numero de grovers 
            shots_list:  list of number of shots
            numebr_grover_list: list of number of Grover operators        

        Return:
            Norac: the total number of oracle calls
    """
    
    Norac = 0
    for k in range(M + 1):
        Nk = shots_list[k]
        mk = number_grover_list[k]
        Norac += Nk * (2 * mk + 1)
            
    return Norac

In [ ]:
def define_shots_list(shot_run, number_grover_list):
    shots_list = []
    for grover in number_grover_list:
        shots_list.append(shot_run)
        
    return shots_list

## Ejecución de nuestro algoritmo en simuladores. 

In [ ]:
final_result_df = pd.DataFrame()
result_df = pd.DataFrame()
results_list = []

error_list = []
shots_list = []  # list of number of shots
number_grover_list = [0, 1, 2, 4]  # list of number of Grover operators
shots_run = [32, 64, 128, 256, 512, 1024]

for shot in shots_run:
    shots_list = define_shots_list(shot, number_grover_list)
    qc_list = create_grover_circuit(number_grover_list, n_bits, b_max)  # list of Grover circuits
    hit_list = run_grover(qc_list, number_grover_list, shots_list, backend)  # list of number of grover operators
    theta_candidate_list, grover_number_list, a_candidate_list = calculate_theta(hit_list, number_grover_list, shots_list)  # list of estimated theta values
    error_list = list(np.abs(mundo_discreto - np.sin(theta_candidate_list)**2)/mundo_discreto)
    result_df = pd.DataFrame({"n_grovers":grover_number_list, "theta_candidate":theta_candidate_list,
                            "a_candidate":a_candidate_list, "rel_error":error_list})
    
    result_df["n_shots"] = shot
    
    final_result_df = pd.concat((final_result_df, result_df), axis=0)

final_result_df.reset_index(drop = True)

In [ ]:
error_0_grover = final_result_df[final_result_df["n_grovers"] == 0]
error_1_grover = final_result_df[final_result_df["n_grovers"] == 1]
error_2_grover = final_result_df[final_result_df["n_grovers"] == 2]
error_4_grover = final_result_df[final_result_df["n_grovers"] == 4]


figure_1 = plt.figure()

#plt.plot(error_0_grover["n_shots"], error_0_grover["rel_error"], "-^")
#plt.plot(error_1_grover["n_shots"], error_1_grover["rel_error"], "-^")
#plt.plot(error_2_grover["n_shots"], error_2_grover["rel_error"], "-^")
plt.plot(error_4_grover["n_shots"], error_4_grover["rel_error"], "-^")

plt.ylim(0, 2)
plt.xlabel("Number of shots")
plt.ylabel("Relative Error")
#plt.legend(["Q = 1", "Q = 2", "Q = 3", "Q = 4"])

plt.show()




In [ ]:
final_result_df = pd.DataFrame()
result_df = pd.DataFrame()
results_list = []

error_list = []
shots_list = []  # list of number of shots
number_grover_list = [0, 1, 2, 4]  # list of number of Grover operators
shots_run = [32, 64, 128, 256, 512, 1024]

n_exec = 10

for ii in range(n_exec):
    for shot in shots_run:
        sys.stdout.write("n_trial=(%d/%d)\r" % ((ii + 1), n_exec))
        sys.stdout.flush()
        shots_list = define_shots_list(shot, number_grover_list)
        qc_list = create_grover_circuit(number_grover_list, n_bits, b_max)  # list of Grover circuits
        hit_list = run_grover(qc_list, number_grover_list, shots_list, backend)  # list of number of grover operators
        theta_candidate_list, grover_number_list, a_candidate_list = calculate_theta(hit_list, number_grover_list, shots_list)  # list of estimated theta values
        error_list = list(np.abs(mundo_discreto - np.sin(theta_candidate_list)**2)/mundo_discreto)
        result_df = pd.DataFrame({"n_grovers":grover_number_list, "theta_candidate":theta_candidate_list,
                                "a_candidate":a_candidate_list, "rel_error":error_list})

        result_df["n_shots"] = shot
        result_df["execc_index"] = ii+1
        result_df["analytical_result"] = mundo_discreto

        final_result_df = pd.concat((final_result_df, result_df), axis=0)

final_result_df.reset_index(drop = True)

In [ ]:
mean_error_1_grover = pd.DataFrame(final_result_df[final_result_df["n_grovers"] == 1].groupby(["n_shots", "n_grovers"])["rel_error"].mean().reset_index())
mean_error_2_grover = pd.DataFrame(final_result_df[final_result_df["n_grovers"] == 2].groupby(["n_shots", "n_grovers"])["rel_error"].mean().reset_index())
mean_error_4_grover = pd.DataFrame(final_result_df[final_result_df["n_grovers"] == 4].groupby(["n_shots", "n_grovers"])["rel_error"].mean().reset_index())
max_error_4_grover = pd.DataFrame(final_result_df[final_result_df["n_grovers"] == 4].groupby(["n_shots", "n_grovers"])["rel_error"].max().reset_index())

figure_1 = plt.figure()

#plt.plot(mean_error_1_grover["n_shots"], mean_error_1_grover["rel_error"], "-^")
#plt.plot(mean_error_2_grover["n_shots"], mean_error_2_grover["rel_error"], "-^")
#plt.plot(mean_error_4_grover["n_shots"], mean_error_4_grover["rel_error"], "-^")
plt.plot(mean_error_4_grover["n_shots"], mean_error_4_grover["rel_error"], "-^")
plt.plot(max_error_4_grover["n_shots"], max_error_4_grover["rel_error"], "-^")


plt.ylim(0,2)
plt.xlabel("Number of shots")
plt.ylabel("Relative Error")
plt.legend(["Relative Error", "Max Error"])

plt.show()
final_result_df.to_csv(r"../Output/MLQAE_vFin_results_REAL_QASM_10.csv")

In [ ]:
for grover_circuit, mlqae_circuit in enumerate(qc_list):
    print("Showing information for: "+str(number_grover_list[grover_circuit]))
    print("Puertas = ", mlqae_circuit.count_ops())
    print("Profundidad / Depth = ", mlqae_circuit.depth())
    print("---- X ----")